# Visualisation

In this Notebook you will look at a variety of visualisation techniques that can be used with the results from SPARQL queries.

You will see how to use the folium package to draw maps and the matplotlib package to draw plots.

## Example 1: Using folium for geographical data

In this example you will revisit a SPARQL query that you met in Notebook 26.1.

First, import the SPARQLWrapper package and set up the usual helper functions.

In [ ]:
# Import the necessary packages
from SPARQLWrapper import SPARQLWrapper, JSON

# Add some helper functions

# A function that will return the results of running a SPARQL query with 
# a defined set of prefixes over a specified endpoint. 
# It follows the same five-step process apart from creating the query, which is
# provided as an argument to the function.
def runQuery(endpoint, prefix, q):
    ''' Run a SPARQL query with a declared prefix over a specified endpoint '''
    sparql = SPARQLWrapper(endpoint)
    sparql.setQuery(prefix+q) # concatenate the strings representing the prefixes and the query
    sparql.setReturnFormat(JSON)
    return sparql.query().convert()
    
# Import pandas to provide facilities for creating a DataFrame to hold results
import pandas as pd

# Function to convert query results into a DataFrame
def dict2df(results):
    ''' A function to flatten the SPARQL query results and return the column values '''
    data = []
    for result in results["results"]["bindings"]:
        tmp = {}
        for el in result:
            tmp[el] = result[el]['value']
        data.append(tmp)

    df = pd.DataFrame(data)
    return df

# Function to run a query and return results in a DataFrame
def dfResults(endpoint, prefix, q):
    ''' Generate a DataFrame containing the results of running
        a SPARQL query with a declared prefix over a specified endpoint 
    '''
    return dict2df(runQuery(endpoint, prefix, q))
        
# Print a limited number of results of a query
def printQuery(results, limit=''):
    ''' Print the results from the SPARQL query '''
    resdata = results["results"]["bindings"]
    if limit!='':
        resdata = results["results"]["bindings"][:limit]
    for result in resdata:
        for ans in result:
            print('{0}: {1}'.format(ans, result[ans]['value']))
        print()

# Run a query and print out a limited number of results
def printRunQuery(endpoint, prefix, q, limit=''):
    ''' Print the results from the SPARQL query '''
    results = runQuery(endpoint, prefix, q)
    printQuery(results, limit)
    
print("Helper functions set up")

The aim is to use the Environment Agency's Bathing Water Linked Data store to find information about bathing water districts within the East Sussex area, including:

* name, latitude and longitude, sediment name and location.

The locations of the districts are found using Ordnance Survey data via a SERVICE request.


In [ ]:
endpoint_envAgency = 'http://environment.data.gov.uk/sparql/bwq/query'

prefix = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX admingeo: <http://statistics.data.gov.uk/def/administrative-geography/>
PREFIX ossr: <http://data.ordnancesurvey.co.uk/ontology/spatialrelations/>
'''

q = '''
SELECT ?location ?districtname ?name ?sedimentname ?lat ?long
WHERE {

    SERVICE <http://data.ordnancesurvey.co.uk/datasets/boundary-line/apis/sparql> { 
        ?area rdfs:label "East Sussex" .
        ?district ossr:within ?area .
        ?district rdfs:label ?districtname .
    }
    
    ?location a <http://environment.data.gov.uk/def/bathing-water/BathingWater> .
    ?location <http://environment.data.gov.uk/def/bathing-water/sedimentTypesPresent> ?sediment .
    ?location <http://statistics.data.gov.uk/def/administrative-geography/district> ?district .
    ?location rdfs:label ?name .
    ?sediment rdfs:label ?sedimentname .
  
     
  
    ?location <http://location.data.gov.uk/def/ef/SamplingPoint/samplingPoint> ?samplingpoint.
    ?samplingpoint <http://www.w3.org/2003/01/geo/wgs84_pos#lat> ?lat.
    ?samplingpoint <http://www.w3.org/2003/01/geo/wgs84_pos#long> ?long.
    FILTER(LANG(?sedimentname) = "" || LANGMATCHES(LANG(?sedimentname), "en"))
    
}
ORDER BY ?districtname
'''

printRunQuery(endpoint_envAgency, prefix, q, limit=5)

The next step is to plot this data on a map using the Python library folium. For this we need the data in the form of a DataFrame.


Convert data to a DataFrame and print out a few rows to see what they look like.

In [ ]:
df = dfResults(endpoint_envAgency, prefix, q)
df[:3]

Import the folium package and define a function that will convert the HTML source of a folium map into a form suitable for displaying in a Python Notebook.

The details of this conversion need not concern you.

In [ ]:
# Helper function for displaying folium maps in a Notebook
import folium
from IPython.display import HTML

# This function builds a folium map suitable for displaying in a Notebook.
# The details of its implementation can be ignored.
# The argument, map, is a folium map object created using folium.Map() - see later for an example.
def inline_map(map):
    '''Embeds the HTML source of the map directly into the IPython Notebook.
       Note: use is made of the HTML5 srcdoc attribute which may not be 
       supported by all browsers.
    '''
    map._build_map()
    srcdoc = map.HTML.replace('"', '&quot;')
    embed = HTML('<iframe srcdoc="{}" '
                 'style="width: 100%; height: 510px; '
                 'border: none"></iframe>'.format(srcdoc))
    return embed


Convert latitude and longitude values to floats for folium. Find mean values of results to centre the map.

In [ ]:
# Cast the lat and long values to floats
df['lat'] = df['lat'].astype(float)
df['long'] = df['long'].astype(float)

# Find their mean values to centre the map
latMean = df['lat'].mean()
longMean = df['long'].mean()

Create the map with appropriate zoom level and centre it. The location argument specifies the centre of the map when it is rendered on the screen. The zoom level indicates the scale of the map: the larger the zoom value, the more detail can be seen but the physical area covered by the map is smaller.

In [ ]:
# Create the map with an appropriate zoom level, and centre it
bathingwater = folium.Map(location=[latMean, longMean], zoom_start=10)

Iterate through the DataFrame, adding each sample point as a marker on the map.

In [ ]:
# Iterate through the DataFrame, adding each sample point as a marker on the map.
# The marker is a simple marker (a circle with a pointer), with centre specified by the location argument.
# When you click on a marker, the text of the popup argument is revealed.
for ix, row in df[['name','lat','long','sedimentname']].iterrows():
    bathingwater.simple_marker(location=[row['lat'], row['long']], popup=row['sedimentname'])

Render the map.

In [ ]:
# Render the map
inline_map(bathingwater)

Folium offers a range of facilities including a variety of types of marker. Markers can have 'pop-ups', small pieces of text that are revealed when the marker is clicked; try it on the map above.

Use the zoom tool in the top left-hand corner of the map to alter the scale of the map.

### Activity 1

Change the value of the `zoom_start` argument in the following code to see what effect this has.

The type of marker has been changed from simple marker to circle marker.

In [ ]:
# Change the value of zoom_start
bathingwater = folium.Map(location=[latMean, longMean], zoom_start=8)

# Calculate the locations of the markers (these change as the value of zoom_start changes)
for ix, row in df[['name','lat','long','sedimentname']].iterrows():
    bathingwater.circle_marker(location=[row['lat'], row['long']], 
                               radius       = 1000,
                               popup        = row['sedimentname'], 
                               line_color   = 'blue',
                               fill_color   = 'blue', 
                               fill_opacity = 0.2)
# Render the map
inline_map(bathingwater)

### Activity 2

Change the arguments to the `circle_marker` method to see the effects.

In [ ]:
bathingwater = folium.Map(location=[latMean, longMean], zoom_start=10)

# Calculate the locations of the markers (these change as the value of zoom_start changes)
for ix, row in df[['name','lat','long','sedimentname']].iterrows():
    bathingwater.circle_marker(location = [row['lat'], row['long']], 
                               radius       = 500, #smaller circles
                               popup        = row['sedimentname'], 
                               line_color   = 'blue',
                               fill_color   = 'yellow', #different colour
                               fill_opacity = 0.3 #different opacity of fill
                               )
# Render the map
inline_map(bathingwater)

Hence, to add markers to a map and render the results, the steps are:

1. Run a query and return the results as a DataFrame (the data should contain latitude and longitude values).

2. Import folium and HTML and set up the `inline_map()` helper function.

3. Calculate the location (latitude, longitude) of the centre of the map and choose an appropriate zoom start level and create a folium map using:

        myMap = folium.Map(location=[latMean, longMean], zoom_start=10)
    
4. Iterate through the DataFrame, adding each sample point as a marker to the map using:

        myMap.circle_marker(...)

    (other types of marker are available).

5. Render the map using the `inline_map()` helper function.


## Example 2: Using the matplotlib package to plot results

Here is another visualisation technique, which uses the matplotlib package.

In this example the aim is to work through the column of sediments in the DataFrame containing the query results, counting the number of each sediment type and then plotting the results.

Start by importing the matplotlib package.

In [ ]:
import matplotlib.pyplot as plt

Use the 'ggplot' style; other styles are available.

(The ggplot style emulates a popular plotting package for 'R', a software environment for statistical computing and graphics.)


In [ ]:
# matplotlib.style.use('ggplot')
plt.style.use('ggplot')

Set up a Series to record the number of each sediment type in the sample. The types of sediment were found in Notebook 26.1, Activity 3. The Series is initialised with counts of zero for each type of sediment.

In [ ]:
s = pd.Series([0,0,0], index=['sand', 'shingle', 'other'])    

Loop through the sediment column.

In [ ]:
# Loop through the sediment 
for ix, row in df[['sedimentname']].iterrows(): 
    if (row['sedimentname'] == 'sand'):
        s['sand'] = s['sand']+1
    elif (row['sedimentname'] == 'shingle'):
        s['shingle'] = s['shingle'] + 1
    else:
        s['other'] = s['other'] + 1
s

Print a bar chart of the sediment counts.

In [ ]:
s.plot(kind='bar')

Note how the plot routine has automatically used the row indexes of the DataFrame to label the horizontal axis. The vertical axis has been automatically scaled for the tallest bar.

Next, render a pie chart of the sediment counts.

In [ ]:
s.plot(kind='pie')

While these have been particularly simple uses of the matplotlib package, they do illustrate how easily plots of data can be rendered.

The plot routine illustrated here has many arguments for tailoring the plots to your requirements (colours, shapes, titles, axes and so forth).


### Activity 3

Extend the sediment column loop to count all sediment types (sand, shingle, rock, marsh, mud, other).

Run the bathing water query again but for a different area (try a county) and plot the results on a pie chart.

In [ ]:
# Insert your solution here

The solution is in the [`26.3solutions`](26.3solutions.ipynb) Notebook.

## Example 3: Visualising house price data

In this example you will examine house price data from the Land Registry. The aim is to compare the prices of the different types of houses in various locations. The results of the query will be displayed on a simple bar chart.

The query finds the prices for different types of house in a single location. Since we want to find the prices in several locations, the query has been embeded in a function with a single argument: the name of the location.

The data held by the Land Registry is in a resource we have labelled `localAuthorityMonth` with a typical value of:

    http://landregistry.data.gov.uk/data/hpi/region/city-of-plymouth/month/2015-01

This references the data for a specific region (local authority), `city-of-plymouth`, for a specific month, January 2015.

In [ ]:
# Define a query to find the house price details of a given location
def query(name):
    s = '''
   SELECT DISTINCT  ?avgDetached ?avgSemi ?avgFlats ?avgTerraced
   WHERE
      {
         # Bind the location of the data for a local authority for the month of January 2015 
         # to the variable ?localAuthorityMonth
         VALUES ?localAuthorityMonth {<http://landregistry.data.gov.uk/data/hpi/region/'''
    s = s + str(name)
    s = s + '''/month/2015-01>}

         # Extract the prices for different types of house 
         ?localAuthorityMonth
            lrhpi:refRegion ?regionURI ;
            lrhpi:averagePricesDetachedSASM ?avgDetached ;
            lrhpi:averagePricesSemiDetachedSASM ?avgSemi ;
            lrhpi:averagePricesFlatMaisonetteSASM ?avgFlats ;
            lrhpi:averagePricesTerracedSASM ?avgTerraced .
    
         # Obtain name of region
         ?regionURI rdfs:label ?regionName .
      FILTER (langMatches(lang(?regionName), "EN") )
   }
   '''
    return s
    

Run the query for three different local authorities.

In [ ]:
endpoint_lr = 'http://landregistry.data.gov.uk/landregistry/query'

prefix = '''
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX lrhpi: <http://landregistry.data.gov.uk/def/hpi/>
'''

q1 = query('city-of-plymouth')
df1 = dfResults(endpoint_lr, prefix, q1)

q2 = query('tyne-and-wear')
df2 = dfResults(endpoint_lr, prefix, q2)

q3 = query('oldham')
df3 = dfResults(endpoint_lr, prefix, q3)

# Concatenate results into a single DataFrame with local authority names as indices
frames = [df1, df2, df3]
df = pd.concat(frames, ignore_index=True)

# Convert the DataFrame to numeric values
df = df.apply(pd.to_numeric)
df.index = ['Plymouth', 'Tyne-and-Wear', 'Oldham']

# Output results
print(df)

In [ ]:
# Plot the results as a bar chart
df.plot(kind='bar')

This output draws your attention to the fact that, in 2015, it was cheaper, on average, to buy a terraced property in Oldham than to buy a flat, which is different from the cases of Plymouth and Tyne-and-Wear. Also, the average price of detached properties was considerably above that of other types of property (at least double) in each of these locations. 

### Activity 4

Repeat the activity in Example 3 with three or four different towns or cities.

In some cases, such as Plymouth, the name of the city is recorded as `city-of-plymouth` to distinguish it from other places with the same name. A list of local authority names and code numbers can be found at the ONS site https://geoportal.statistics.gov.uk/geoportal/catalog/content/filelist.page?redirect. However, it can be quite tricky to discover the name used for the local authority of a well-known town or city. You might like to try the Ordnance Survey's linked data site at http://data.ordnancesurvey.co.uk/ to view named places.

In [ ]:
# Insert your solution here

The solution is in the [`26.3solutions`](26.3solutions.ipynb) Notebook.

### Activity 5

Find the average house price indices for detached houses in `city-of-plymouth` for each month of 2014.

Plot the results on a line graph. In matplotlib a simple line graph (a series of points joined by straight lines) is easily obtained by using

    plot(x,y)

where `x` and `y` specify the coordinates of the points: `x` contains all the x coordinates and `y` contains all the y coordinates such that, for example, `(x[0], y[0])` is the first point.


In [ ]:
# Insert your solution here

The solution is in the [`26.3solutions`](26.3solutions.ipynb) Notebook.

## Summary

This Notebook has illustrated the use of two visualisation techniques: one for maps (folium) and one for plots (matplotlib).

Using folium, you saw how to:

1. identify part of the globe via latitude and longitude and zoom level
2. add location markers to a map
3. render a map.

The data, which included latitude and longitude values, that was added to the map was extracted via a SPARQL query.

Using matplotlib, you saw how to:

1. add data to a DataFrame
2. call the `plot()` function against the DataFrame to create a plot
3. identify the kind of plot required using the `kind` argument to `plot()`
4. draw a line graph.

The data used to create a plot came from two sources:

1. the Environment Agency's bathing water linked data store
2. the Land Registry's house price data

which were searched using SPARQL queries. The data returned from the SPARQL queries had to be transformed into a format suitable for plotting.

## What next?

If you are working through this Notebook as part of an inline exercise, return to the module materials now.

If you are working through this set of Notebooks as a whole, you've completed the Part 26 Notebooks.